In [1]:
import numpy as np
import pandas as pd

In [2]:
import yahoo_historical
MSFT = yahoo_historical.Fetcher('MSFT', [2014, 7, 1], [2019, 7, 1]).getHistorical()
AAPL = yahoo_historical.Fetcher('AAPL', [2014, 7, 1], [2019, 7, 1]).getHistorical()
GOOGL = yahoo_historical.Fetcher('GOOGL', [2014, 7, 1], [2019, 7, 1]).getHistorical()

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM

Using TensorFlow backend.


In [4]:
MSFT.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-07-01,41.860001,42.150002,41.689999,41.869999,37.121700,26917000
1,2014-07-02,41.730000,41.900002,41.529999,41.900002,37.148304,20208100
2,2014-07-03,41.910000,41.990002,41.560001,41.799999,37.059635,15969300
3,2014-07-07,41.750000,42.119999,41.709999,41.990002,37.228085,21952400
4,2014-07-08,41.869999,42.000000,41.610001,41.779999,37.041901,31218200


In [5]:
window = 100
data1 = []
data2 = []
data3 = []
y_data=[]
for i in range(MSFT.shape[0] - window - 1):
    data1.append(np.array(MSFT.iloc[i:i+window, 5]) / MSFT.iloc[i, 5])
    data2.append(np.array(AAPL.iloc[i:i+window, 5]) / AAPL.iloc[i, 5])
    data3.append(np.array(GOOGL.iloc[i:i+window, 5]) / GOOGL.iloc[i, 5])
    y_data.append(np.array(MSFT.iloc[i+window+1, 5]) / MSFT.iloc[i, 5])
    

In [6]:
data4=[]
for i in range(len(data1)):
    data4.append(np.transpose(np.array([data1[i],data2[i],data3[i]])))

In [7]:
np.array(data1).shape

(1157, 100)

In [8]:
np.array(data4).shape

(1157, 100, 3)

In [9]:
train_size = int(0.9 * len(data4))
x_train = np.array(data4[:train_size])
x_test = np.array(data4[train_size:])
y_train=np.array(y_data[:train_size])
y_test=np.array(y_data[train_size:])
np.random.seed(2019)
np.random.shuffle(x_train)
np.random.seed(2019)
np.random.shuffle(y_train)
#x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
#x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [10]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 3))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 3))

In [11]:
def build_model(layers):
    model = Sequential()

    model.add(LSTM(input_shape = (layers[1], layers[0]), output_dim = layers[1], return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(layers[2], return_sequences = False))
    model.add(Dropout(0.2))

    model.add(Dense(output_dim = layers[3]))
    model.add(Activation("linear"))

    model.compile(loss = "mse", optimizer = "adam")
    return model

In [12]:
model = build_model([3, 100, 500, 1])

C:\Users\DidaFENG\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(100, 3), return_sequences=True, units=100)`
  after removing the cwd from sys.path.
C:\Users\DidaFENG\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
model.fit(x_train, y_train, batch_size = 64, epochs = 20, validation_split = 0.1, verbose = 1)

Train on 936 samples, validate on 105 samples
Epoch 1/20
936/936 [==============================] - 29s 31ms/step - loss: 0.3685 - val_loss: 0.0084
Epoch 2/20
936/936 [==============================] - 27s 29ms/step - loss: 0.0237 - val_loss: 0.0149
Epoch 3/20
936/936 [==============================] - 28s 30ms/step - loss: 0.0101 - val_loss: 0.0090
Epoch 4/20
936/936 [==============================] - 28s 30ms/step - loss: 0.0069 - val_loss: 0.0047
Epoch 5/20
936/936 [==============================] - 28s 30ms/step - loss: 0.0063 - val_loss: 0.0041
Epoch 6/20
936/936 [==============================] - 28s 30ms/step - loss: 0.0054 - val_loss: 0.0037
Epoch 7/20
936/936 [==============================] - 29s 31ms/step - loss: 0.0052 - val_loss: 0.0034
Epoch 8/20
936/936 [==============================] - 29s 31ms/step - loss: 0.0049 - val_loss: 0.0031
Epoch 9/20
936/936 [==============================] - 29s 31ms/step - loss: 0.0051 - val_loss: 0.0030
Epoch 10/20
936/936 [===============

In [ ]:
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train Score: %.4f MSE (%.4f RMSE)' % (train_score, np.sqrt(train_score)))
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test Score: %.4f MSE (%.4f RMSE)' % (test_score, np.sqrt(test_score)))

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(y_pred, color = 'red', label = 'prediction')
plt.plot(y_test, color = 'blue', label = 'true value')
plt.legend()
plt.show()

In [ ]:
y_test_diff = (y_test[1:] - y_test[:-1] > 0).astype(int)
y_pred_diff = (y_pred[1:,0] - y_pred[:-1,0] > 0).astype(int)

In [ ]:
from sklearn import metrics

print('confusion matrix:')
print(metrics.confusion_matrix(y_test_diff, y_pred_diff))
print(' accuracy :', metrics.accuracy_score(y_test_diff, y_pred_diff))
print('precision :', metrics.precision_score(y_test_diff, y_pred_diff))
print('   recall :', metrics.recall_score(y_test_diff, y_pred_diff))

In [ ]:
model.summary()

(1). The target ticker is MSFT. <br/>
(2). I chose AAPL, GOOGL and MSFT to explain target ticker. <br/>
(3). I set window as 100 days and used 3 tickers. So my input is a $3\times100$ matrix. The first layer is LTSM layer as input layer and the second layer with 500 neurons as hidden layer. I also used the Dense layer as my output layer.<br/>
(4). I have 1244101 free parameters. <br/>
(5). Linear function to train model. <br/>
(6). Adam, an algorithm for first-order gradient-based optimization of stochastic objective functions, based on adaptive estimates of lower-order moments. <br/>
(7). See the graph above.<br/>